In [3]:
import numpy as np
import sys
import os
from pathlib import Path
notebook_path = Path(os.getcwd())
sys.path.append(str(notebook_path.parent))
import scale.olm as olm

2023-08-18 17:43:29 [info     ] Initialized scale.olm


In [10]:
obiwan='/Applications/SCALE-6.3.0.app/Contents/Resources/bin/obiwan'
f71='/Users/ww5/Downloads/e10v50.f71'

In [11]:
hist = olm.common.get_history_from_f71(obiwan,f71,-1)

2023-08-18 17:49:49 [info     ] extracting history from /Users/ww5/Downloads/e10v50.f71
2023-08-18 17:49:49 [info     ] running command:
/Applications/SCALE-6.3.0.app/Contents/Resources/bin/obiwan view -format=info /Users/ww5/Downloads/e10v50.f71
2023-08-18 17:49:50 [info     ]    pos         time        power         flux      fluence       energy    initialhm libpos   case   step DCGNAB
2023-08-18 17:49:50 [info     ]    (-)          (s)         (MW)    (n/cm2-s)      (n/cm2)        (MWd)      (MTIHM)    (-)    (-)    (-)    (-)
2023-08-18 17:49:50 [info     ]      1  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  5.45574e-01      1      1      0 DC----
2023-08-18 17:49:50 [info     ]      2  1.72800e+05  2.33249e+01  3.15984e+14  5.46021e+19  4.66499e+01  5.45574e-01      2      1      1 DC----
2023-08-18 17:49:50 [info     ]      3  4.32000e+06  2.33321e+01  3.19120e+14  1.37806e+21  1.16659e+03  5.45574e-01      3      1      2 DC----
2023-08-18 17:49:50 [info   

2023-08-18 17:49:50 [info     ]     45  7.12800e+07  8.44659e+00  3.48002e+14  2.37741e+22  6.97255e+03  2.12124e-01     14     11     13 DC----
2023-08-18 17:49:50 [info     ]     46  7.77600e+07  8.44541e+00  3.50893e+14  2.60479e+22  7.60596e+03  2.12124e-01     15     11     14 DC----
2023-08-18 17:49:50 [info     ]     47  8.42400e+07  8.44417e+00  3.53795e+14  2.83404e+22  8.23927e+03  2.12124e-01     16     11     15 DC----
2023-08-18 17:49:50 [info     ]     48  9.07200e+07  8.44289e+00  3.56705e+14  3.06519e+22  8.87249e+03  2.12124e-01     17     11     16 DC----
2023-08-18 17:49:50 [info     ]     49  9.72000e+07  8.44157e+00  3.59625e+14  3.29823e+22  9.50561e+03  2.12124e-01     18     11     17 DC----
2023-08-18 17:49:50 [info     ]     50  1.03680e+08  8.44021e+00  3.62547e+14  3.53316e+22  1.01386e+04  2.12124e-01     19     11     18 DC----
2023-08-18 17:49:50 [info     ]     51  1.10160e+08  8.43882e+00  3.65482e+14  3.76999e+22  1.07715e+04  2.12124e-01     20     11

2023-08-18 17:49:50 [info     ]     93  1.81440e+08  7.05485e+00  3.97114e+14  6.49065e+22  1.44855e+04  1.96884e-01     31     21     30 DC----
2023-08-18 17:49:50 [info     ]     94  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  4.54181e-02      1     31      0 DC----
2023-08-18 17:49:50 [info     ]     95  1.72800e+05  1.39694e+00  3.14427e+14  5.43329e+19  2.79388e+00  4.54181e-02      2     31      1 DC----
2023-08-18 17:49:50 [info     ]     96  4.32000e+06  1.39544e+00  3.16869e+14  1.36845e+21  6.97751e+01  4.54181e-02      3     31      2 DC----
2023-08-18 17:49:50 [info     ]     97  8.64000e+06  1.39782e+00  3.19569e+14  2.74899e+21  1.39666e+02  4.54181e-02      4     31      3 DC----
2023-08-18 17:49:50 [info     ]     98  1.29600e+07  1.40058e+00  3.21805e+14  4.13918e+21  2.09695e+02  4.54181e-02      5     31      4 DC----
2023-08-18 17:49:50 [info     ]     99  1.94400e+07  1.40221e+00  3.24022e+14  6.23884e+21  3.14860e+02  4.54181e-02      6     31

2023-08-18 17:49:51 [info     ]    188  1.72800e+05  3.99422e+01  3.15308e+14  5.44852e+19  7.98845e+01  1.00000e+00      2     -2      1 DC----
2023-08-18 17:49:51 [info     ]    189  4.32000e+06  3.99422e+01  3.18294e+14  1.37451e+21  1.99711e+03  1.00000e+00      3     -2      2 DC----
2023-08-18 17:49:51 [info     ]    190  8.64000e+06  3.99417e+01  3.21087e+14  2.76161e+21  3.99419e+03  1.00000e+00      4     -2      3 DC----
2023-08-18 17:49:51 [info     ]    191  1.29600e+07  3.99413e+01  3.23290e+14  4.15822e+21  5.99126e+03  1.00000e+00      5     -2      4 DC----
2023-08-18 17:49:51 [info     ]    192  1.94400e+07  3.99410e+01  3.25805e+14  6.26944e+21  8.98684e+03  1.00000e+00      6     -2      5 DC----
2023-08-18 17:49:51 [info     ]    193  2.59200e+07  3.99404e+01  3.28712e+14  8.39950e+21  1.19824e+04  1.00000e+00      7     -2      6 DC----
2023-08-18 17:49:51 [info     ]    194  3.24000e+07  3.99398e+01  3.31574e+14  1.05481e+22  1.49779e+04  1.00000e+00      8     -2

In [12]:
for pb in hist['burndata']:
    print("cycle{{ power={} burn={} }}".format(pb['power'],pb['burn']))
    

cycle{ power=39.9422 burn=2.0 }
cycle{ power=39.9422 burn=48.0 }
cycle{ power=39.9417 burn=50.0 }
cycle{ power=39.9413 burn=50.0 }
cycle{ power=39.941 burn=75.0 }
cycle{ power=39.9404 burn=75.0 }
cycle{ power=39.9398 burn=75.0 }
cycle{ power=39.9392 burn=75.0 }
cycle{ power=39.9385 burn=75.0 }
cycle{ power=39.9379 burn=75.0 }
cycle{ power=39.9372 burn=75.0 }
cycle{ power=39.9365 burn=75.0 }
cycle{ power=39.9358 burn=75.0 }
cycle{ power=39.9351 burn=75.0 }
cycle{ power=39.9343 burn=75.0 }
cycle{ power=39.9336 burn=75.0 }
cycle{ power=39.9328 burn=75.0 }
cycle{ power=39.932 burn=75.0 }
cycle{ power=39.9312 burn=75.0 }
cycle{ power=39.9304 burn=75.0 }
cycle{ power=39.9295 burn=75.0 }
cycle{ power=39.9287 burn=75.0 }
cycle{ power=39.9278 burn=75.0 }
cycle{ power=39.9269 burn=75.0 }
cycle{ power=39.926 burn=75.0 }
cycle{ power=39.9251 burn=75.0 }
cycle{ power=39.9242 burn=75.0 }
cycle{ power=39.9233 burn=75.0 }
cycle{ power=39.9223 burn=75.0 }
cycle{ power=39.9214 burn=75.0 }
